In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# Import the requests library.
import requests

# IMport the API key
from config import weather_api_key
from config import g_key
import gmaps

In [3]:
# Store the CSV you saved created in part one into a DataFrame
city_data_df = pd.read_csv("weather_data/WeatherPy_vacation.csv")
city_data_df.head()

,City_ID,City,Country,Weather Description,Max Temp,Lat,Lng,Hotel Name
0,3,Hilo,US,clear sky,75.20,19.73,-155.09,Hilo Hawaiian Hotel
1,4,Vaini,TO,few clouds,77.00,-21.20,-175.20,Keleti Beach Resort
2,10,Kahului,US,few clouds,75.20,20.89,-156.47,Maui Seaside Hotel
3,11,Garowe,SO,scattered clouds,81.55,8.41,48.48,Curubo Hotel
4,12,Kapaa,US,few clouds,77.00,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach


In [4]:
# Create separate DataFrames for each city
yarada_data = city_data_df.loc[city_data_df['City'] == 'Yarada']
narasannapeta_data = city_data_df.loc[city_data_df['City'] == 'Narasannapeta']
tarabha_data = city_data_df.loc[city_data_df['City'] == 'Tarabha']
angul_data = city_data_df.loc[city_data_df['City'] == 'Angul']

In [5]:
yarada_data

,City_ID,City,Country,Weather Description,Max Temp,Lat,Lng,Hotel Name
97,385,Yarada,IN,broken clouds,84.2,17.65,83.27,"D-26/2, Dolphin Hill,"


In [6]:
narasannapeta_data

,City_ID,City,Country,Weather Description,Max Temp,Lat,Lng,Hotel Name
126,510,Narasannapeta,IN,scattered clouds,81.1,18.42,84.05,G.lingaraju home


In [7]:
tarabha_data

,City_ID,City,Country,Weather Description,Max Temp,Lat,Lng,Hotel Name
127,511,Tarabha,IN,few clouds,86.16,20.73,83.65,tulapada full


In [8]:
angul_data

,City_ID,City,Country,Weather Description,Max Temp,Lat,Lng,Hotel Name
129,515,Angul,IN,clear sky,81.63,20.85,85.1,Bluebay 08 satkosia Tarava


In [9]:
import gmaps.datasets

In [10]:
gmaps.configure(api_key=g_key)

In [11]:
#yarada = yarada_data.set_index['Lat']
#yarada.head()

In [12]:
# Turn the Lat adn Lng into tuples
yarada_tup = list(yarada_data[['Lat', 'Lng']].itertuples(index=False, name=None))
narasannapeta_tup = list(narasannapeta_data[['Lat', 'Lng']].itertuples(index=False, name=None))
tarabha_tup = list(tarabha_data[['Lat', 'Lng']].itertuples(index=False, name=None))
angul_tup = list(angul_data[['Lat', 'Lng']].itertuples(index=False, name=None))

In [13]:
# Create directional figure
fig = gmaps.figure()
yarada2angul = gmaps.directions_layer(yarada_tup[0], angul_tup[0])
fig.add_layer(yarada2angul)
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Create directional figure for all cities. 
fig = gmaps.figure()
yarada2angul = gmaps.directions_layer(yarada_tup[0], angul_tup[0], 
                                      waypoints = [narasannapeta_tup[0], tarabha_tup[0]], 
                                      travel_mode='DRIVING')
fig.add_layer(yarada2angul)
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
four_cities = [yarada_data, narasannapeta_data, tarabha_data, angul_data]
four_cities

[    City_ID    City Country Weather Description  Max Temp    Lat    Lng  \
 97      385  Yarada      IN       broken clouds      84.2  17.65  83.27   
 
                Hotel Name  
 97  D-26/2, Dolphin Hill,  ,
      City_ID           City Country Weather Description  Max Temp    Lat  \
 126      510  Narasannapeta      IN    scattered clouds      81.1  18.42   
 
        Lng        Hotel Name  
 126  84.05  G.lingaraju home  ,
      City_ID     City Country Weather Description  Max Temp    Lat    Lng  \
 127      511  Tarabha      IN          few clouds     86.16  20.73  83.65   
 
         Hotel Name  
 127  tulapada full  ,
      City_ID   City Country Weather Description  Max Temp    Lat   Lng  \
 129      515  Angul      IN           clear sky     81.63  20.85  85.1   
 
                      Hotel Name  
 129  Bluebay 08 satkosia Tarava  ]

In [16]:
# Merge all the DataFrames into one.
four_cities = pd.concat(four_cities)

In [17]:
four_cities

,City_ID,City,Country,Weather Description,Max Temp,Lat,Lng,Hotel Name
97,385,Yarada,IN,broken clouds,84.20,17.65,83.27,"D-26/2, Dolphin Hill,"
126,510,Narasannapeta,IN,scattered clouds,81.10,18.42,84.05,G.lingaraju home
127,511,Tarabha,IN,few clouds,86.16,20.73,83.65,tulapada full
129,515,Angul,IN,clear sky,81.63,20.85,85.10,Bluebay 08 satkosia Tarava


In [18]:
# Tell Google what we want to show in the marker
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp}</dd>
</dl>
"""

# Store the DataFrame Row.
city_info = [info_box_template.format(**row) for index, row in four_cities.iterrows()]

In [19]:
# Add markers for each city the traveler will be going to. 
locations = four_cities[["Lat", "Lng"]]
fig = gmaps.figure(center=(19.5, 84.5), zoom_level=7)
marker_layer = gmaps.marker_layer(locations, info_box_content=city_info)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))